### Testing API

In [21]:
# base_url = 'https://api.pushshift.io/reddit/'


# def request_posts(subreddit, days_ago, base_url=base_url,
#                   endpoint='search/submission/', is_video='is_video=false'):
#     try:
#         response = requests.get(
#             f'{base_url}{endpoint}?subreddit={subreddit}&{is_video}&before={days_ago}d&after={days_ago+1}d&size=1000')
#         assert response.status_code == 200
#     except:
#         pass

#     return response


# def make_requests(subreddit, days_of_data):
#     all_results = []

#     for i in range(1, days_of_data):
#         try:
#             entry = request_posts(subreddit, i)
#             all_results.append(pd.DataFrame(entry.json()['data']))
#         except:
#             pass
#         if i % 20 == 0:
#             print(f'{i} of {days_of_data} requests completed')
#         time.sleep(1.5)

#     return pd.concat(all_results)


# def request_all_subs(list_of_subreddits, days_of_data):
#     all_results = []
#     for sub in list_of_subreddits:
#         print(f'Querying {sub}...')
#         sub_df = make_requests(sub, days_of_data)
#         all_results.append(sub_df)
#     return pd.concat(all_results)


# def main():
#     subs = ['asoiaf', 'freefolk', 'gameofthrones']
#     df = request_all_subs(subs, 100)
#     df.to_csv('./data/subreddit_data.csv')


# if __name__ == "__main__":
#     main()

### Validating API results

In [22]:
# # This script accepts the raw API response .csv and outputs one with NAs and deleted posts removed,
# # plus any HTML fragments removed 

# # Imports
# from bs4 import BeautifulSoup
# import pandas as pd
# import warnings

# # Ignore warnings
# warnings.filterwarnings("ignore")

# # Import Data
# df = pd.read_csv('../data/subreddit_data.csv')

# print('Number of Entries per subreddit pre-cleaning:')
# print(df['subreddit'].value_counts())

# # Drop posts without selftext
# has_self = (df[~df['selftext'].isna()])
# has_self = has_self[has_self['domain'].str.contains('self')]

# # Drop posts where text has been removed or deleted by users/moderators
# has_self = has_self[has_self['selftext']!='[removed]']
# has_self = has_self[has_self['selftext']!='[deleted]']
# has_self = has_self[~has_self['selftext'].str.startswith('[removed]')]
# has_self = has_self[~has_self['selftext'].str.startswith('[deleted]')]

# # Select the columns we're interested in modeling
# columns = ['author', 'full_link', 'id', 'num_comments', 'num_crossposts', 'over_18',
#                      'spoiler', 'subreddit', 'title', 'total_awards_received', 'upvote_ratio',
#                      'edited', 'gilded','selftext']

# has_self = has_self[columns]

# # Reset index of results because it is annoying 
# has_self.reset_index(drop=True, inplace=True)

# print('Number of Entries per subreddit post-cleaning:')
# print(has_self['subreddit'].value_counts())

# # Just in case there are any HTML impurities in the raw data, we will use BeautifulSoup 
# # to clean the text columns
# for col in has_self:
#     if has_self[col].dtypes == 'O':
#         for item in has_self.index:
#             has_self.loc[item,col] = BeautifulSoup(has_self.loc[item,col]).get_text()

# has_self.to_csv('../data/cleaned_subreddit_data.csv', index=False)

### Investigating our full data

In [23]:
#from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from scipy.stats import uniform, loguniform
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

#import warnings

In [24]:
df = pd.read_csv('../data/cleaned_subreddit_data.csv')

In [25]:
df.shape

(71555, 7)

In [26]:
df.dropna(inplace=True)

In [27]:
df.shape

(70985, 7)

In [28]:
# df = df.drop(df[df['author'] == 'AutoModerator'].index)

#  df.loc[:,'selftext'] = df['selftext'].str.replace('x200B','')

#  df.loc[:,'selftext'] = df['selftext'].str.replace('\n',' ')

# df = df[~df['selftext'].isna()]

# df.reset_index(inplace=True)

In [29]:
df.columns

Index(['level_0', 'index', 'id', 'selftext', 'author', 'title', 'subreddit'], dtype='object')

In [30]:
df.shape

(70985, 7)

In [31]:
# most_common_authors = df.author.value_counts().sort_values(ascending=False).head(21).index[1:]

# table = df.loc[df['author'].isin(most_common_authors),['author','subreddit','id']].groupby(['author','subreddit']).count()

# table.index.levels

# ax = table.unstack().plot.barh(figsize=(10,10))


In [32]:
# tfid_reddit = TfidfVectorizer(lowercase=True, stop_words='english',
#                               ngram_range=(1,2),max_df=.95, min_df=3, max_features=5000)

# vect_table = tfid_reddit.fit_transform(df['selftext'])

# vect_df = pd.DataFrame(vect_table.toarray(), columns = tfid_reddit.get_feature_names())

# # vect_df_sorted = vect_df.sum().sort_values()

# # #vect_df_sorted.head(100)

# # vect_df_sorted.tail(100)

# vect_df['subreddit'] = df['subreddit']
# #vect_df['selftext'] = df['selftext']

# # vect_df[vect_df['subreddit'] == 'freefolk'].drop(columns='subreddit').sum().sort_values(ascending=False).head(100)

# # vect_df[vect_df['subreddit'] == 'gameofthrones'].drop(columns='subreddit').sum().sort_values(ascending=False).head(100)

# # vect_df[vect_df['subreddit'] == 'asoiaf'].drop(columns='subreddit').sum().sort_values(ascending=False).head(100)

# vect_df.isna().sum().sort_values(ascending=False).head(20)

In [33]:
df.isna().sum().sort_values(ascending=False).head(20)

subreddit    0
title        0
author       0
selftext     0
id           0
index        0
level_0      0
dtype: int64

In [34]:
df.loc[45,'selftext']

'Like yes I know you get Theons internal monologues and such is the book but I feel hes a lot more sympathetic and emotional in the show Alfie Allen does a fantastic job and you really see the conflict in his acting and his regret for what hes done Maester Luwin too i feel is a lot better and Rodriks death is definitely more impactful in the show than the book'

In [35]:
df['selftext'].isna().sum()

0

In [36]:
df['title'].isna().sum()

0

Preprocessing workflow:

- tokenize
- remove stopwords
- lemmatize


In [37]:
df['selftext'] = [word_tokenize(str(text)) for text in df['selftext']]

df['title'] = [word_tokenize(str(text)) for text in df['title']]

In [38]:
lemmatizer = WordNetLemmatizer()

In [39]:
lemmatizer.lemmatize('churches')

'church'

In [40]:
df['selftext'] = [[lemmatizer.lemmatize(word) for word in text] for text in df['selftext']]

In [41]:
stemmer = PorterStemmer()

In [42]:
df['selftext'] = [' '.join([stemmer.stem(word) for word in text]) for text in df['selftext']]

In [43]:
df.selftext[0]

'for me it is jon snow He use to be my favorit but the overpand to hi charact by say he is the subject to everi propheci and is basic jesu is so annoy He is still a good charact especi in the book but think jon snow will defeat the other then he will kill dani and then rule wise and justli becaus he is the perfect hero is annoy'

### Testing Modeling process

In [44]:
sample = df.sample(frac=.05)

In [45]:
sample.subreddit.value_counts()

asoiaf           1579
freefolk          990
gameofthrones     980
Name: subreddit, dtype: int64

In [46]:
sample.subreddit.value_counts(normalize=True)

asoiaf           0.444914
freefolk         0.278952
gameofthrones    0.276134
Name: subreddit, dtype: float64

In [47]:
sample.shape

(3549, 7)

In [48]:
X = sample['selftext']
y = sample['subreddit']

X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, random_state=42)

In [49]:
# tfid_final_params = {'tfid__max_df': 0.325,
#   'tfid__max_features': 6500,
#   'tfid__min_df': 5,
#   'tfid__ngram_range': (1, 2)}


#### Baseline independent models: LogisticRegression and DecisionTree

##### Logistic Regression

In [136]:
logit_pipe = Pipeline([
    ('tfid',TfidfVectorizer(max_df=.325,
                           max_features = 6500,
                           min_df = 5,
                           ngram_range = (1,2))),
    ('logit',LogisticRegression(solver='liblinear'))
])

logit_params = {
    'logit__C':loguniform(.01,50),
    'logit__penalty':['l2','l1',None],
}

In [137]:
logit_model = RandomizedSearchCV(logit_pipe,
                  logit_params,
                  n_iter = 75,
                  n_jobs=-1,
                  cv = 3,
                  verbose=1)

In [138]:
logit_model.fit(X_train,y_train)

Fitting 3 folds for each of 75 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  1.7min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('tfid',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=0.325,
                                                              max_features=6500,
                                                              min_df=5,
                                                      

In [139]:
logit_model.best_params_

{'logit__C': 2.552049865944663, 'logit__penalty': 'l2'}

In [140]:
cross_val_score(logit_model.best_estimator_,X_train,y_train, cv=5)

array([0.56554307, 0.58988764, 0.57677903, 0.57035647, 0.57973734])

In [141]:
cross_val_score(logit_model.best_estimator_,X_test,y_test, cv=5)

array([0.5505618 , 0.61797753, 0.61235955, 0.58988764, 0.53370787])

In [142]:
logit_model.score(X_test,y_test) - logit_model.score(X_train,y_train)

-0.2984409480652932

In [143]:
logit_model.best_estimator_.score(X_test,y_test) - logit_model.best_estimator_.score(X_train,y_train)

-0.2984409480652932

In [144]:
logit_model.best_estimator_.score(X_train,y_train)

0.9186656671664168

In [145]:
logit_model.best_estimator_.score(X_test,y_test)

0.6202247191011236

##### Decision Tree

In [111]:
tree_pipe = Pipeline([
    ('tfid',TfidfVectorizer(max_df=.325,
                           max_features = 6500,
                           min_df = 5,
                           ngram_range = (1,2))),
    ('tree',DecisionTreeClassifier())
])

tree_params = {
    'tree__max_depth':uniform(1,20),
    'tree__min_samples_split':uniform(0,.5),
    'tree__min_samples_leaf':uniform(0,.5),
}

In [112]:
tree_model = RandomizedSearchCV(tree_pipe,
                  tree_params,
                  n_iter = 100,
                  n_jobs=-1,
                  cv = 3,
                  verbose=1)

In [113]:
tree_model.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   20.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('tfid',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=0.325,
                                                              max_features=6500,
                                                              min_df=5,
                                                      

In [114]:
tree_model.best_params_

{'tree__max_depth': 18.788855431067592,
 'tree__min_samples_leaf': 0.0064049947690571485,
 'tree__min_samples_split': 0.33934451621277056}

In [115]:
cross_val_score(tree_model.best_estimator_,X_train,y_train, cv=5)

array([0.47565543, 0.52621723, 0.52059925, 0.50281426, 0.48405253])

In [116]:
cross_val_score(tree_model.best_estimator_,X_test,y_test, cv=5)

array([0.44382022, 0.47752809, 0.52808989, 0.43820225, 0.41573034])

In [117]:
tree_model.score(X_test,y_test) - tree_model.score(X_train,y_train)

-0.057350538214039015

In [119]:
tree_model.best_estimator_.score(X_test,y_test) - tree_model.best_estimator_.score(X_train,y_train)

-0.057350538214039015

In [123]:
tree_model.best_estimator_.score(X_train,y_train)

0.5629685157421289

In [124]:
tree_model.best_estimator_.score(X_test,y_test)

0.5056179775280899

#### Next step: implement voting classifier

In [ ]:
#Voting Classifier:
# LogisticRegression
# MultinomialNB
# GradientBoostingClassifier
# AdaBoostClassifier
# RandomForestClassifier
# DecisionTreeClassifier
# SVC


# Pipeline (tfid => Voting Classifier)

# Random 

5.869304461753987